In [1]:
import mysql.connector
import re


def clean_domain(d):
    d = d.strip().lower()
    d = re.sub(r'[;!?()]', '', d)
    return d


def process_mbox(file_name):
    domain_count = {}
    with open(file_name, 'r') as file:
        for l in file:
            if l.startswith('From '):
                email_address = l.split()[1]
                domain_name = email_address.split('@')[-1]
                domain_name = clean_domain(domain_name)
                if domain_name in domain_count:
                    domain_count[domain_name] += 1
                else:
                    domain_count[domain_name] = 1

    return domain_count


def store_in_database(domain_counts):
    connection = mysql.connector.connect(
        host="localhost",
        user="root",    
        password="Root@123",
        database="Eduonix_Project" 
    )
    cursor = connection.cursor()
    cursor.execute('DROP TABLE IF EXISTS Counts')
    cursor.execute('''
        CREATE TABLE Counts (
            org VARCHAR(255), 
            count INT
        )
    ''')

    for domain, count in domain_counts.items():
        cursor.execute('INSERT INTO Counts (org, count) VALUES (%s, %s)', (domain, count))

    connection.commit()
    connection.close()

def display_results():
    connection = mysql.connector.connect(
        host="localhost",
        user="root",    
        password="Root@123",
        database="Eduonix_Project"
    )
    cursor = connection.cursor()

    cursor.execute('SELECT org, count FROM Counts ORDER BY count DESC')

    print("Counts:")
    for row in cursor:
        print(f"{row[0]} {row[1]}")

    connection.close()


def main():
    file_name = input("Enter file name: ")
    domain_counts = process_mbox(file_name)
    store_in_database(domain_counts)
    display_results()

main()


Enter file name: mbox.txt
Counts:
iupui.edu 8
umich.edu 7
uct.ac.za 6
media.berkeley.edu 4
caret.cam.ac.uk 1
gmail.com 1
